# 簡単なサンプル

In [24]:

import requests
from bs4 import BeautifulSoup
import csv


url = "https://finance.yahoo.com/quote/4755.T/history/?period1=1714435200&period2=1745971200"
ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)" \
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36" \
        "Edg/125.0.0.0"

res = requests.get(url, headers={'User-Agent': ua})

# レスポンスの HTML から BeautifulSoup オブジェクトを作る
soup = BeautifulSoup(res.text, 'lxml')
table = soup.findAll("table", {"class":"hideOnPrint"})[0]
rows = table.findAll("tr")

with open("ebook1s.csv", "w", encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
    for row in rows:
        # Date Open High Low Close Adj Close Volume
        csvRow = []
        for cell in row.findAll(['td']):
            csvRow.append(cell.get_text())
        if any(csvRow):  # 空の行をスキップ
            writer.writerow(csvRow)


/var/folders/nv/rp02rh3n4w54kp2_phnp7nt80000gn/T/ipykernel_3188/769449739.py:15: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = soup.findAll("table", {"class":"hideOnPrint"})[0]
/var/folders/nv/rp02rh3n4w54kp2_phnp7nt80000gn/T/ipykernel_3188/769449739.py:16: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  rows = table.findAll("tr")
/var/folders/nv/rp02rh3n4w54kp2_phnp7nt80000gn/T/ipykernel_3188/769449739.py:24: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for cell in row.findAll(['td']):


In [26]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import re
import os

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)" \
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36" \
        "Edg/125.0.0.0"
DATA_DIR = "../data"

def validate_date_format(date_str: str) -> bool:
  """
  日付文字列がYYYY-MM-DD形式かどうかを検証する
  
  Args:
      date_str (str): 検証する日付文字列
      
  Returns:
      bool: 有効な形式の場合はTrue、そうでない場合はFalse
  """
  pattern = r'^\d{4}-\d{2}-\d{2}$'
  if not re.match(pattern, date_str):
    return False
  
  try:
    datetime.strptime(date_str, '%Y-%m-%d')
    return True
  except ValueError:
    return False

def get_stock_csv(code: str, from_date: str, to_date: str):
  """
  株価データを取得し、CSVファイルに保存する

  Args:
      code (str): 株価データを取得する銘柄コード
      from_date (str): 株価データを取得する開始日
      to_date (str): 株価データを取得する終了日

  Raises:
      ValueError: 日付の形式が不正です。YYYY-MM-DD形式で入力してください。
  """
  if not validate_date_format(from_date) or not validate_date_format(to_date):
    raise ValueError("日付の形式が不正です。YYYY-MM-DD形式で入力してください。")
  
  if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
  
  from_date_epoch = int(datetime.strptime("{}T09:00:00".format(from_date), "%Y-%m-%dT%H:%M:%S").timestamp())
  to_date_epoch = int(datetime.strptime("{}T00:00:00".format(to_date), "%Y-%m-%dT%H:%M:%S").timestamp())
  
  url = "https://finance.yahoo.com/quote/{}/history/?period1={}&period2={}".format(code, from_date_epoch, to_date_epoch)

  file_name = "{}/stock_data_{}_{}_{}.csv".format(DATA_DIR, code, from_date, to_date)
  
  res = requests.get(url, headers={'User-Agent': USER_AGENT})
  rows = BeautifulSoup(res.text, 'lxml').find_all("table", {"class":"hideOnPrint"})[0].find_all("tr")
  
  with open(file_name, "w", encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])  
    writer.writerows((
        row for row in map(
            lambda row: [cell.get_text() for cell in row.find_all(['td'])], rows)
        if any(row))
    )


In [25]:
fromDate = "2025-04-01"
toDate = "2025-04-30"
code = "4755.T"

get_stock_csv(code, fromDate, toDate)